# 实用python编程系列：数据建模

本节以**titanic乘客存活率预测任务**为例，介绍如何利用pandas和sklearn进行数据建模。

## 数据透视表 Pivot table
pandas.pivot_table方法可以为一个特定的DataFrame建立一个数据透视表。

In [31]:
import numpy as np
import pandas as pd
data = {"A": ["foo", "foo", "foo", "foo", "foo", "bar", "bar", "bar", "bar"],
        "B": ["one", "one", "one", "two", "two", "one", "one", "two", "two"],
        "C": ["small", "large", "large", "small", "small", "large", "small", "small", "large"],
        "D": [1, 2, 2, 3, 3, 4, 5, 6, 7]}
df = pd.DataFrame(data)
df

,A,B,C,D
0,foo,one,small,1
1,foo,one,large,2
2,foo,one,large,2
3,foo,two,small,3
4,foo,two,small,3
5,bar,one,large,4
6,bar,one,small,5
7,bar,two,small,6
8,bar,two,large,7


In [2]:
table = pd.pivot_table(df, values='D', index=['A', 'B'], columns=['C'], aggfunc=np.mean)
table

C        large  small
A   B                
bar one    4.0    5.0
    two    7.0    6.0
foo one    2.0    1.0
    two    NaN    3.0

In [36]:
table = pd.pivot_table(df, values='D', index=['A'],aggfunc=[np.sum, np.mean])
table

,sum,mean
,D,D
A,,
bar,22,5.5
foo,11,2.2


In [3]:
table = pd.pivot_table(df, values='D', index=['A', 'B'], columns=['C'], aggfunc=[np.sum, np.mean])
table

sum        mean      
C       large small large small
A   B                          
bar one   4.0   5.0   4.0   5.0
    two   7.0   6.0   7.0   6.0
foo one   4.0   1.0   2.0   1.0
    two   NaN   6.0   NaN   3.0

## 用pivot table回答作业4

In [4]:
df = pd.read_excel('data/titanic/train.xlsx')
table = pd.pivot_table(df, values="Survived", index=['Pclass', 'Sex'], aggfunc=np.mean)
table

Survived
Pclass Sex             
1      female  0.968085
       male    0.368852
2      female  0.921053
       male    0.157407
3      female  0.500000
       male    0.135447

在不同港口登船是否对存活率有影响？

In [5]:
table = pd.pivot_table(df, values="Survived", index=['Pclass', 'Sex'], columns=['Embarked'], aggfunc=np.mean)
table

Embarked              C         Q         S
Pclass Sex                                 
1      female  0.976744  1.000000  0.958333
       male    0.404762  0.000000  0.354430
2      female  1.000000  1.000000  0.910448
       male    0.200000  0.000000  0.154639
3      female  0.652174  0.727273  0.375000
       male    0.232558  0.076923  0.128302

## 预测哪些乘客能活下来

读取测试数据

In [6]:
testset = pd.read_excel('data/titanic/test_with_label.xlsx')
print testset.shape
print testset.Sex.value_counts()

(418, 12)
male      266
female    152
Name: Sex, dtype: int64


基于规则，如果是女性，则survived=1, 否则survived=0

In [7]:
def naive_predict(x):
    return x.Sex == 'female'

res = testset.apply(naive_predict, axis=1)
matched = testset.Survived == res
accuracy = sum(matched) / float(testset.shape[0])
print "Accuracy : %s" % "{0:.3%}".format(accuracy)

Accuracy : 76.555%


进一步，分别查看幸存者、遇难者的预测准确率

In [8]:
count = testset.Survived.value_counts()
for s in [0, 1]:
    correct = (testset.Survived == s) & matched
    print s, count[s], sum(correct), sum(correct)/float(count[s])

0 260 214 0.823076923077
1 158 106 0.670886075949


## 利用训练数据“学习”决策过程

首先读取训练数据，并检查是否存在数据缺失的情况

In [9]:
df = pd.read_excel('data/titanic/train.xlsx')
df.apply(lambda x: sum(x.isnull()), axis=0)

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

### 填充缺失值

DataFrame提供了fillna方法填充缺失。

为了避免对原始数据误操作，我们首先创建一个新的DataFrame

In [10]:
trainset = df.copy(deep=True)

我们先观察一下缺Embarked值的乘客样本。

In [11]:
df[df.Embarked.isnull()]

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
61,62,1,1,"Icard, Miss. Amelie",female,38.0,0,0,113572,80.0,B28,NaN
829,830,1,1,"Stone, Mrs. George Nelson (Martha Evelyn)",female,62.0,0,0,113572,80.0,B28,NaN


这两个样本都是女性 ($Sex=female$)，且来自头等舱($Pclass=1$)。

注意到Embarked的取值是3个字母 $\{C, Q, S\}$，无法求平均值或中位值。

考虑用多数值替换缺失值。

In [12]:
df[(df.Pclass==1) & (df.Sex=='female')].Embarked.value_counts()

S    48
C    43
Q     1
Name: Embarked, dtype: int64

多数乘客从S港登船。因此，我们假定这两个乘客也是从S港等船。

为了直接对new_df进行修改，需要将fillna方法的inplace参数设为True。

In [13]:
trainset['Embarked'].fillna('S', inplace=True)

检查是否填充成功。

In [14]:
trainset[df['Embarked'].isnull()]

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
61,62,1,1,"Icard, Miss. Amelie",female,38.0,0,0,113572,80.0,B28,S
829,830,1,1,"Stone, Mrs. George Nelson (Martha Evelyn)",female,62.0,0,0,113572,80.0,B28,S


接下来，填充缺失的Age值。

用pivot_table统计男女乘客年龄的中位值

In [18]:
age_table = pd.pivot_table(df, values='Age', index=['Sex'], aggfunc=np.median)
print age_table.loc['female', 'Age'], age_table.loc['male', 'Age']

 27.0 29.0


使用fillna方法将缺失值替换为中位值。

因为中位值因性别而异，所以不能像之前一样直接给一个常数。

In [19]:
def estimate_age(x):
    return age_table.loc[x['Sex'], 'Age']

new_values = df[df['Age'].isnull()].apply(estimate_age, axis=1)
trainset['Age'].fillna(new_values, inplace=True)

检查Age是否填充成功

In [20]:
trainset[df['Age'].isnull()].head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
5,6,0,3,"Moran, Mr. James",male,29.0,0,0,330877,8.4583,NaN,Q
17,18,1,2,"Williams, Mr. Charles Eugene",male,29.0,0,0,244373,13.0000,NaN,S
19,20,1,3,"Masselmani, Mrs. Fatima",female,27.0,0,0,2649,7.2250,NaN,C
26,27,0,3,"Emir, Mr. Farred Chehab",male,29.0,0,0,2631,7.2250,NaN,C
28,29,1,3,"O'Dwyer, Miss. Ellen ""Nellie""",female,27.0,0,0,330959,7.8792,NaN,Q


In [21]:
df[df['Age'].isnull()].head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
5,6,0,3,"Moran, Mr. James",male,NaN,0,0,330877,8.4583,NaN,Q
17,18,1,2,"Williams, Mr. Charles Eugene",male,NaN,0,0,244373,13.0000,NaN,S
19,20,1,3,"Masselmani, Mrs. Fatima",female,NaN,0,0,2649,7.2250,NaN,C
26,27,0,3,"Emir, Mr. Farred Chehab",male,NaN,0,0,2631,7.2250,NaN,C
28,29,1,3,"O'Dwyer, Miss. Ellen ""Nellie""",female,NaN,0,0,330959,7.8792,NaN,Q


检查缺失的Age是否被填充了?

In [22]:
trainset.apply(lambda x: sum(x.isnull()), axis=0)

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age              0
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         0
dtype: int64

至此，除了Cabin属性，trainset的缺失值都已经被填充上了。

## 分类

1. 特征提取
2. 分类器训练
3. 预测
4. 评分

### 特征提取

选择若干观察量作为分类依据。

#### 处理分类变量（Categorical variable）

分类变量是指以有限几个数或符号作为取值范围的变量。本例中的分类变量有Pclass, Sex和Embarked

In [23]:
trainset['SexCode'] = trainset.Sex == 'female'
trainset['c1'] = trainset.Pclass == 1
trainset['c2'] = trainset.Pclass == 2
trainset['c3'] = trainset.Pclass == 3

trainset['C'] = trainset.Embarked == 'C'
trainset['Q'] = trainset.Embarked == 'Q'
trainset['S'] = trainset.Embarked == 'S'

在测试集上重复此操作

In [24]:
testset['SexCode'] = testset.Sex == 'female'
testset['c1'] = testset.Pclass == 1
testset['c2'] = testset.Pclass == 2
testset['c3'] = testset.Pclass == 3

testset['C'] = testset.Embarked == 'C'
testset['Q'] = testset.Embarked == 'Q'
testset['S'] = testset.Embarked == 'S'

### 构建分类器

使用sklearn机器学习包

In [25]:
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics

训练一个随机森林(RandomForest)分类器

In [26]:
features = str.split('SexCode c1 c2 c3 C Q S')
target = 'Survived'

model = RandomForestClassifier(n_estimators=100)
model.fit(trainset[features], trainset[target])

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

接着，我们在训练集上测试性能。
这个是为了检查代码是否有问题。
如果在训练集上的分类准确率很低，就不要指望在未知的测试集上能有好的表现。

In [27]:
predictions = model.predict(trainset[features])
accuracy = metrics.accuracy_score(predictions, trainset[target])
print "Accuracy : %s" % "{0:.3%}".format(accuracy)

Accuracy : 81.145%


### 测试
现在，在测试集上使用训练好的模型进行预测

In [28]:
predictions = model.predict(testset[features])

### 评分

In [29]:
accuracy = metrics.accuracy_score(predictions, testset[target])
print "Accuracy : %s" % "{0:.3%}".format(accuracy)

Accuracy : 77.751%
